# Importing Dependencies

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U keras>=3
!pip install -U tensorflow-text

In [ ]:
import jax

jax.devices()

In [ ]:
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation
# overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

# Loading model

In [ ]:
import keras
import keras_nlp

In [ ]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices())

`LayoutMap` from the distribution API specifies how the weights and tensors should be sharded or replicated, using the string keys, for example, `token_embedding/embeddings` below, which are treated like regex to match tensor paths. Matched tensors are sharded with model dimensions (8 TPUs); others will be fully replicated.

In [ ]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded on 8 TPUs
layout_map["token_embedding/embeddings"] = (None, model_dim)
# Regex to match against the query, key and value matrices in the decoder
# attention layers
layout_map["decoder_block.*attention.*(query|key|value).*kernel"] = (
    None, model_dim, None)

layout_map["decoder_block.*attention_output.*kernel"] = (
    None, None, model_dim)
layout_map["decoder_block.*ffw_gating.*kernel"] = (model_dim, None)
layout_map["decoder_block.*ffw_linear.*kernel"] = (None, model_dim)

`ModelParallel` allows you to shard model weights or activation tensors across all devcies on the `DeviceMesh`. In this case, some of the Gemma 7B model weights are sharded across 8 TPU chips according the `layout_map` defined above. Now load the model in the distributed way.

In [ ]:
model_parallel = keras.distribution.ModelParallel(
    device_mesh, layout_map, batch_dim_name="batch")

keras.distribution.set_distribution(model_parallel)
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_7b_en")

Now verify that the model has been partitioned correctly. Let's take `decoder_block_1` as an example.

In [ ]:
decoder_block_1 = gemma_lm.backbone.get_layer('decoder_block_1')
print(type(decoder_block_1))
for variable in decoder_block_1.weights:
  print(f'{variable.path:<58}  {str(variable.shape):<16}  {str(variable.value.sharding.spec)}')

# Reading Training Dataset

In [ ]:
import pandas as pd
import numpy as np
import re
import random

In [ ]:
train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
train.head()

## Visualization Of Training Dataset

In [ ]:
train.size ###This will give the total number of elements (rows * columns) in DataFrame. 

In [ ]:
train.shape ### We want the number of rows or columns separately

In [ ]:
# Display information about the train DataFrame
train.info()

In [ ]:
train.describe() # Generate descriptive statistics of the train DataFrame

#### We have 17307 essays to train, but we will first try using few shot prompting.

# Reading Test Dataset

In [ ]:
test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
test.head()

## Visualization Of Test Dataset

In [ ]:
test.size ###This will give the total number of elements (rows * columns) in DataFrame. 


In [ ]:
test.shape ### We want the number of rows or columns separately

In [ ]:
test.info() # Display information about the train DataFrame


In [ ]:
test.describe() # Generate descriptive statistics of the train DataFrame

# Prompt
#### So that it gives better answer.
#### We are using Few Shot Prompting Technique
##### Which means we are using all the training dataset to give it in prompt as examples 

In [ ]:
prompt = """You are English Professor, an exceptionally intelligent Professor tasked with score english essay.

English Professor will be given a english essay, and will provide a precise score, adhering to the following rules:
- English Professor guarantees to give a precise score, always within the range of 1 to 6.
- Score will be concise and limited to a single number.
- Only the final result will be provided, no additional information.
- The score written in the "Score" section must be concise and only include the final result.
- English Professor will always follow these rules.


{examples}


# English Essay
{essay}

# Answer (only one number between 0 and 999)
"""

In [ ]:
def out(model, examples_df: pd.DataFrame | None , df, template):
    submission = {"essay_id": [], "score": []}

    examples = ""
    if examples_df is not None and not examples_df.empty:
        examples = []
        for idx, row in examples_df.iterrows():
            examples.append("# English Essay")
            examples.append(str(row["full_text"]))
            examples.append("# Score (only one number between 1 and 6)")
            examples.append(str(row["score"]))
        examples = "\n".join(examples)
    
    for idx, row in df.iterrows():
        try:
            model_input = prompt.format(examples=examples, essay=row["full_text"])

            output = gemma_lm.generate(prompt,max_length=3)
            
            output = int(re.sub(r"[^0-9]", "", output))

            submission["essay_id"].append(row["essay_id"])
            submission["score"].append(output)
        except Exception as e:
            submission["essay_id"].append(row["essay_id"])
            submission["score"].append(random.randint(1, 6))
            
    submission_df = pd.DataFrame(submission)
    submission_df["score"] = submission_df["score"].apply(lambda x: abs(x) % 1000)
    return submission_df

The `out` function generates submissions based on provided examples, a main DataFrame, a template, and a machine learning model. Here's how it works:

1. **Input Parameters:**
   - `model`: The machine learning model used to generate answers.
   - `examples_df`: A DataFrame containing example questions and answers (optional).
   - `df`: The main DataFrame containing questions.
   - `template`: A template string used to format questions and generate answers.

2. **Example Formatting:**
   - If `examples_df` is provided and not empty, the function formats it into a string.

3. **Submission Generation:**
   - For each row in the main DataFrame (`df`):
     - It attempts to generate an answer using the provided template and model.
     - If successful, it appends the answer to the submission dictionary.
     - If an exception occurs during answer generation, it prints the exception and adds a random answer instead.

4. **Final Submission DataFrame:**
   - The function converts the submission dictionary into a DataFrame.
   - It ensures that the answer values are within the range of 1 to 6".

5. **Output:**
   - The function returns the submission DataFrame containing question IDs and their respective answers.


## We will trying with 10 examples in prompt

In [ ]:
tr = train.iloc[:10] # Get the first row of the train DataFrame
# tr_df = tr.to_frame().T # Convert the tr variable into a DataFrame with a single row
tr

In [ ]:
output = out(gemma_lm, tr, test, prompt)
output


# Into CSV

In [ ]:
output.to_csv("submission.csv", index=False)

## To Do In Future
* Finetune the llm using train data
* Do better Visualization